objective: To compare two faces'

General Approach
1. Face Detection
2. Alignment
3. Face Description
4. Face Recognition



First Approach:  Face Detection By Cascade Classifier and face similarity using MSE and SSIM

In [3]:
### Importing the Packages

import cv2
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import numpy as np


In [4]:
def mse(imageA, imageB):
  err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
  err= float(imageA.shape[0] * imageA.shape[1])
  print(err)
  return err


def compare_images(imageA, imageB, title):
  mt = mse(imageA, imageB)
  st = ssim(imageA, imageB)
	# setup the figure
  fig = plt.figure(title)
  plt.suptitle("MSE: %.2f, SSIM: %.2f" % (mt, st))
	# show first image
  ax = fig.add_subplot(1, 2, 1)
  plt.imshow(imageA, cmap = plt.cm.gray)
  plt.axis("off")
	# show the second image
  ax = fig.add_subplot(1, 2, 2)
  plt.imshow(imageB, cmap = plt.cm.gray)
  plt.axis("off")
  plt.show()
  return([mt,st])

In [9]:
files=["/content/4x6.jpg","/content/Ananya_photo.JPG","/content/ankit_photo_4x6.jpg","/content/mummy_adhar.png",
"/content/mummy_photo.png","/content/tunnu_photo.jpeg"]

import itertools
files_com=[]
for i, j in itertools.combinations_with_replacement(files, 2):
  tiff=[i,j]
  files_com.append(tiff)

In [10]:
len(files_com)

21

In [11]:
def resize_img(img1,img2):
  width, height = img1.shape[0],img1.shape[1]
  width_2, height_2=img2.shape[0],img2.shape[1]
  if width>=width_2 or height>=height_2:
    img1= cv2.resize(img1,(img2.shape[1],img2.shape[0]))
  else:
    img2= cv2.resize(img2,(img1.shape[1],img1.shape[0]))
  #plt.imshow(img2)
  plt.imshow(img1)
  return([img1,img2])


In [ ]:
error_t=[]
for i in files_com:
  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

  # images to grayscale
  img1=cv2.imread(i[0])
  img2=cv2.imread(i[1])
  gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
  gray2= cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

  gray=[gray1,gray2]

  # detect faces in the 2 images
  faces1 = face_cascade.detectMultiScale(gray1, 1.3, 5)
  faces2 = face_cascade.detectMultiScale(gray2, 1.3, 5)
  roi_gray=[]
  roi_color=[]

  size=gray1.shape

  # crop out only the face of the first and second images
  for (x,y,w,h) in faces1:

      extra=int(w/6)
      x1=max(0,x-extra)
      y1=max(0,y-extra)
      x2=min(size[1],x1+2*extra+w)
      y2=min(size[0],y1+2*extra+w)

      img1 = cv2.rectangle(img1,(x1,y1),(x2-1,y2-1),(0,0,255),4)
      roi_gray .append(gray1[y1:y2, x1:x2])
      roi_color .append(img1[y1:y2, x1:x2])

  if len(faces1)==0:
    roi_gray .append(gray1)
    roi_color .append(img1)

  size=gray2.shape
  for (x,y,w,h) in faces2:

      extra=int(w/6)
      x1=max(0,x-extra)
      y1=max(0,y-extra)
      x2=min(size[1],x1+2*extra+w)
      y2=min(size[0],y1+2*extra+w)

      img2 = cv2.rectangle(img2,(x1,y1),(x2-1,y2-1),(0,0,255),4)
      roi_gray .append(gray2[y1:y2, x1:x2])
      roi_color .append(img2[y1:y2, x1:x2])

  if len(faces2)==0:
    roi_gray .append(gray2)
    roi_color .append(img2)

  # img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
  # roi_color=cv2.cvtColor(roi_color,cv2.COLOR_BGR2RGB)

  # plot the cropped out grayscale images of the originals
  plt.figure(figsize=(5,5))
  plt.imshow(roi_gray[0],cmap='gray', vmin=0, vmax=255)
  plt.title('ROI of image 1')
  plt.show()

  plt.figure(figsize=(5,5))
  plt.imshow(roi_gray[1],cmap='gray', vmin=0, vmax=255)
  plt.title('ROI of image 2')
  plt.show()

  img1_sizet=resize_img(roi_gray[0],roi_gray[1])[0]
  print(img1_sizet.shape)
  img2_sizet=resize_img(roi_gray[0],roi_gray[1])[1]
  print(img2_sizet.shape)
  plt.imshow(img2_sizet)
  com=compare_images(img1_sizet,img2_sizet,"MSE and SSIM")
  tz=[i[0],i[1],com[0],com[1]]
  error_t.append(tz)








In [26]:
import pandas as pd
dfj=pd.DataFrame(error_t)

In [27]:
dfj.columns=["Image1","Image2","MSE","SSIM"]

In [20]:
dfj.to_csv("Different_Similarity_measures_for_face_match",index=False)

Approach 2: Face Recogintion

In [2]:
pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=9fef2733dcfad290cc07fcb23a7a4e0692fd40d02f6d175649bf419d997c9748
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
import face_recognition
face_reco=[]
for i in files_com:
  img_modi = face_recognition.load_image_file(i[0])
  img_modi = cv2.cvtColor(img_modi,cv2.COLOR_BGR2RGB)
  #------to find the face location
  face = face_recognition.face_locations(img_modi)[0]
  print(face)
  #--Converting image into encodings
  train_encode = face_recognition.face_encodings(img_modi)[0]
  #----- lets test an image
  test = face_recognition.load_image_file(i[1])
  test = cv2.cvtColor(test, cv2.COLOR_BGR2RGB)
  test_encode = face_recognition.face_encodings(test)[0]
  face_test=face_recognition.face_locations(test)[0]
  tz=face_recognition.compare_faces([train_encode],test_encode)
  print(face_recognition.compare_faces([train_encode],test_encode))
  cv2.rectangle(img_modi, (face[3], face[0]),(face[1], face[2]), (255,0,255), 1)
  cv2.rectangle(test, (face_test[3], face_test[0]),(face_test[1], face_test[2]), (255,0,255), 1)
  from google.colab.patches import cv2_imshow
  cv2_imshow(img_modi)
  cv2_imshow(test)
  tf=[i[0],i[1],tz]
  face_reco.append(tf)


In [24]:
import pandas as pd
face_reco_df=pd.DataFrame(face_reco)

In [29]:
face_reco_df.columns=["Image1","Image2","Face_recognition_model"]

In [30]:
dfj_t=pd.merge(dfj,face_reco_df,on=["Image1","Image2"],how="left")

In [32]:
dfj_t.to_csv("Face_Match_measures.csv",index=False)

Approach 3:  Using SIFT algorithm

In [ ]:
sift_t=[]
for i in files_com:
  print(i[0],i[1])
  img1=cv2.imread(i[0])
  img2 = cv2.imread(i[1])

  # img1=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
  # img2=cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

  # convert the images from bgr to rgb
  img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
  img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)

  face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

  # images to grayscale
  gray1 = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
  gray2= cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

  gray=[gray1,gray2]

  # detect faces in the 2 images
  faces1 = face_cascade.detectMultiScale(gray1, 1.1, 5)
  faces2 = face_cascade.detectMultiScale(gray2, 1.1, 5)
  roi_gray=[]
  roi_color=[]

  size=gray1.shape

  # crop out only the face of the first and second images
  for (x,y,w,h) in faces1:

      extra=int(w/6)
      x1=max(0,x-extra)
      y1=max(0,y-extra)
      x2=min(size[1],x1+2*extra+w)
      y2=min(size[0],y1+2*extra+w)

      img1 = cv2.rectangle(img1,(x1,y1),(x2-1,y2-1),(0,0,255),4)
      roi_gray .append(gray1[y1:y2, x1:x2])
      roi_color .append(img1[y1:y2, x1:x2])

  if len(faces1)==0:
    roi_gray .append(gray1)
    roi_color .append(img1)

  size=gray2.shape
  for (x,y,w,h) in faces2:

      extra=int(w/6)
      x1=max(0,x-extra)
      y1=max(0,y-extra)
      x2=min(size[1],x1+2*extra+w)
      y2=min(size[0],y1+2*extra+w)

      img2 = cv2.rectangle(img2,(x1,y1),(x2-1,y2-1),(0,0,255),4)
      roi_gray .append(gray2[y1:y2, x1:x2])
      roi_color .append(img2[y1:y2, x1:x2])

  if len(faces2)==0:
    roi_gray .append(gray2)
    roi_color .append(img2)

  sift = cv2.SIFT_create()

  kp1, des1 = sift.detectAndCompute(roi_gray[0],None)
  kp2, des2 = sift.detectAndCompute(roi_gray[1],None)

  # create a bruteforce matcher
  bf = cv2.BFMatcher()

  matches=bf.knnMatch(des1,des2,k=2)


  good = []
  for m,n in matches:
    #print(m.distance,n.distance)
    if m.distance < 0.75*n.distance:
      good.append([m])

  img3 = cv2.drawMatchesKnn(roi_gray[0],kp1,roi_gray[1],kp2,good,None,flags=2)

  # decide whether the images are a match or not based on the number of good matches.
  # Yes, crude but a good starting intuition
  if len(good)>=30:
    z="It's a Match"
    print("It's a Match")

  else:
    z="Not a Match"
    print("Not a Match")

  tef=[i[0],i[1],z]
  sift_t.append(tef)

  plt.figure(figsize=(40,20))
  plt.imshow(img3)
  plt.show()



In [34]:
sift_t

[['/content/4x6.jpg', '/content/4x6.jpg', "It's a Match"],
 ['/content/4x6.jpg', '/content/Ananya_photo.JPG', 'Not a Match'],
 ['/content/4x6.jpg', '/content/ankit_photo_4x6.jpg', 'Not a Match'],
 ['/content/4x6.jpg', '/content/mummy_adhar.png', 'Not a Match'],
 ['/content/4x6.jpg', '/content/mummy_photo.png', 'Not a Match'],
 ['/content/4x6.jpg', '/content/tunnu_photo.jpeg', 'Not a Match'],
 ['/content/Ananya_photo.JPG', '/content/Ananya_photo.JPG', "It's a Match"],
 ['/content/Ananya_photo.JPG', '/content/ankit_photo_4x6.jpg', 'Not a Match'],
 ['/content/Ananya_photo.JPG', '/content/mummy_adhar.png', 'Not a Match'],
 ['/content/Ananya_photo.JPG', '/content/mummy_photo.png', 'Not a Match'],
 ['/content/Ananya_photo.JPG', '/content/tunnu_photo.jpeg', 'Not a Match'],
 ['/content/ankit_photo_4x6.jpg',
  '/content/ankit_photo_4x6.jpg',
  "It's a Match"],
 ['/content/ankit_photo_4x6.jpg', '/content/mummy_adhar.png', 'Not a Match'],
 ['/content/ankit_photo_4x6.jpg', '/content/mummy_photo.pn

In [35]:
SIFT_t=pd.DataFrame(sift_t)

In [36]:
SIFT_t.columns=["Image1","Image2","SIFT_MATCH"]

In [38]:
dfj_t=pd.merge(dfj_t,SIFT_t,on=["Image1","Image2"],how="left")

In [40]:
dfj_t.to_csv("Face_Match_measures_new.csv",index=False)

Using DeepFace Algorithm

In [41]:
pip install deepface

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 11.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 10.7 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116932 sha256=d20594617be77151ca537833e1f137cbc930a0dd6607bd8e1f83324299dcc5cf
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [43]:

from deepface import DeepFace
models = [
  "VGG-Face",
  "Facenet",
  "Facenet512",
  "OpenFace",
  "DeepFace",
  "ArcFace",
  "Dlib"
]

deep=[]
for j in range(len(models)):
  for i in files_com:
    img1=i[0]
    img2=i[1]
    obj=DeepFace.verify(img1,img2,model_name = models[j])
    tef=[models[j],i[0], i[1], obj['verified']]
    deep.append(tef)

arcface_weights.h5  will be downloaded to  /root/.deepface/weights/arcface_weights.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/arcface_weights.h5
To: /root/.deepface/weights/arcface_weights.h5
100%|██████████| 137M/137M [00:00<00:00, 223MB/s]
Downloading...
From: http://dlib.net/files/dlib_face_recognition_resnet_model_v1.dat.bz2
To: /root/.deepface/weights/dlib_face_recognition_resnet_model_v1.dat.bz2


dlib_face_recognition_resnet_model_v1.dat is going to be downloaded


100%|██████████| 21.4M/21.4M [00:00<00:00, 33.5MB/s]


In [45]:
deepface=pd.DataFrame(deep)

In [46]:
deepface.to_csv("DeepFace_results.csv",index=False)